In [11]:
import sys
sys.path.append("../")  # 添加项目根目录到路径中

In [12]:
import torch
import torch.optim as optim
from torch import nn

import hgp
from hgp.models import HGNNP,CHGNN
from hgp.function import StraightThroughEstimator_1, StraightThroughEstimator

import numpy as np
import random
DEVICE = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
DEVICE


device(type='cuda', index=1)

In [13]:
seed = 42
torch.manual_seed(seed) # 为CPU设置随机种子
torch.cuda.manual_seed(seed) # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU，为所有GPU设置随机种子
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.	
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [14]:
from hgp.models import ParameterDict

# fmt: off
h_hyper_prmts = ParameterDict()
l_hyper_prmts = ParameterDict()

partitions = 2

weight = 29
limit = 0.006
sub = 0.006
lr = 3e-5

h_hyper_prmts["convlayers11"] = {"in_channels": 1290, "out_channels": 512, "use_bn": True, "drop_rate": 0.2}
h_hyper_prmts["convlayers14"] = {"in_channels":512, "out_channels": 512, "use_bn": True, "drop_rate": 0.05}
h_hyper_prmts["convlayers141"] = {"in_channels": 512, "out_channels": 512, "use_bn":True, "drop_rate": 0.05}
h_hyper_prmts["convlayers144"] = {"in_channels": 512, "out_channels": 256, "use_bn": True, "drop_rate": 0.05}
h_hyper_prmts["convlayers143"] = {"in_channels": 256, "out_channels": 256, "use_bn": True, "drop_rate": 0.05}

l_hyper_prmts["linerlayer0"] = {"in_channels":256, "out_channels":256, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer31"] = {"in_channels":256, "out_channels":3, "use_bn":True, "drop_rate":0.05}

hyper = {
    "h_hyper_prmts": h_hyper_prmts,
    "l_hyper_prmts":l_hyper_prmts,
    "init_features_dim":list(h_hyper_prmts.values())[0]["in_channels"],
    "partitions":partitions
}

# fmt: on

In [15]:
def loss_bs_matrix(outs, hg, device,weight):
    # fmt: off
    r"""
    对于超图的损失函数的矩阵形式.
    
    Args:
        ``outs``(`torch.nn.Module`):  模型的输出. Size :math:`(N, nums_classes)`.   
        ``hg``(`Hypergraph`):  超图对象.  
    """
    # fmt: on
    H = hg.H.to_dense().to(device)
    outs = outs.to(device)
    nn = torch.matmul(outs, (1 - torch.transpose(outs, 0, 1)))
    ne_k = torch.matmul(nn, H)
    ne_k = ne_k.mul(H)

    H_degree = torch.sum(H, dim=0)
    H_degree = H_degree

    H_1 = ne_k / H_degree
    a2 = 1 - H_1
    a3 = torch.prod(a2, dim=0)
    a3 = a3.sum()
    loss_1 = -1 * a3

    # pun = torch.mul(ne_k, H)

    # loss_1 = pun.sum()
    loss_2 = torch.var(torch.sum(outs, dim=0)).to(device)
    loss = weight * loss_1 + loss_2
    return loss, loss_1, loss_2

#### 定义用于训练的类Trainer

In [16]:
from hgp.function import Scale
class Trainer(nn.Module):
    # fmt: off
    r"""
    用于承担训练的类.
    ---
    Args:
        ``net``: (``torch.nn.Module``): 网络模型.  
        ``X``: (``torch.Tensor``): 作为输入的顶点特征矩阵. Size :math:`(N, C_{in})`.  
        ``hg``: (``dhg.Hypergraph``): 包含 :math:`N` 个顶点的超图结构.  
    """
    # fmt: on
    def __init__(self, net, X, hg, optimizer):
        super().__init__()
        self.X: torch.Tensor = X.to(DEVICE)
        self.hg = hg.to(DEVICE)
        self.de = self.hg.H.to_dense().sum(dim=0).to("cpu").to(DEVICE)
        self.optimizer: torch.optim.Optimizer = optimizer
        self.layers = nn.ModuleList()
        self.layers.append(net.to(DEVICE))
        self.weight = 200
        self.scale = Scale()
    def forward(self, X):
        X = self.layers[0](X, self.hg)
        # X = self.scale.apply(X,200)
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    def run(self, epoch):
        self.train()  # train mode | 设置为训练模式
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss, loss_1, loss_2 = loss_bs_matrix(outs, self.hg, device=DEVICE,weight=self.weight)
        loss.backward()
        self.optimizer.step()

        return loss.item(), loss_1.item(), loss_2.item()

#### 准备数据

In [17]:
import dhg 
from dhg.data import Cooking200,HouseCommittees
data = HouseCommittees()
e_list = data["edge_list"]
num_v = data["num_vertices"]
G = dhg.Hypergraph(data["num_vertices"],data["edge_list"])
num_v, data["num_edges"]

(1290, 341)

In [18]:
# X = torch.randn(size=(G.num_v, hyper["init_features_dim"]))
X = torch.eye(hyper["init_features_dim"])
net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
hgnn_trainer = Trainer(net=net, X=X, hg=G, optimizer=None).to(DEVICE)
for (k,v) in hyper["l_hyper_prmts"].items():
    hgnn_trainer.layers.append(nn.BatchNorm1d(num_features=v["in_channels"]).to(DEVICE)) if v["use_bn"] else None
    hgnn_trainer.layers.append(nn.ReLU().to(DEVICE))
    if v["drop_rate"] > 0:
        hgnn_trainer.layers.append(nn.Dropout(v["drop_rate"]))
    hgnn_trainer.layers.append(nn.Linear(in_features=v["in_channels"],out_features=v["out_channels"],device=DEVICE))
hgnn_trainer.layers.append(nn.Softmax(dim=1))

ModuleList(
  (0): HGNNP(
    (layers): ModuleList(
      (0): HGNNPConv(
        (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.2, inplace=False)
        (theta): Linear(in_features=1290, out_features=512, bias=True)
      )
      (1): HGNNPConv(
        (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.05, inplace=False)
        (theta): Linear(in_features=512, out_features=512, bias=True)
      )
      (2): HGNNPConv(
        (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.05, inplace=False)
        (theta): Linear(in_features=512, out_features=512, bias=True)
      )
      (3): HGNNPConv(
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

In [19]:
# hgnn_trainer.layers
# for n,p in hgnn_trainer.named_parameters():
#     print(n,p)
hgnn_trainer.weight = weight

In [20]:
temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
optim1 = optim.Adam(hgnn_trainer.parameters(), lr=lr, weight_decay=5e-8)
hgnn_trainer.optimizer = optim1
for epoch in range(20000):
    if hgnn_trainer.weight > limit:
        hgnn_trainer.weight = hgnn_trainer.weight - sub
    loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
    temp_loss_total += loss
    temp_loss1 += loss_1
    temp_loss2 += loss_2
    if epoch % 1 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 1}")
        print(f"                , loss1: {temp_loss1.item() / 1}")
        print(f"                , loss2: {temp_loss2.item() / 1}")
        print(f"                , weight: {hgnn_trainer.weight}")
        print(f"=================================")
        # sys.stdout.flush()
        temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
    hgnn_trainer.train()
    if loss_1 < -80 and loss_2 < 2 and StraightThroughEstimator.apply(hgnn_trainer.forward(hgnn_trainer.X)).sum().item() == 1290:
        break
        

in 0 epoch, average loss: 7234.71044921875
                , loss1: -1.031557559967041
                , loss2: 7264.61962890625
                , weight: 28.994
in 1 epoch, average loss: 3475.697265625
                , loss1: -0.9419052600860596
                , loss2: 3503.001220703125
                , weight: 28.988
in 2 epoch, average loss: 1721.377685546875
                , loss1: -1.0025029182434082
                , loss2: 1750.4322509765625
                , weight: 28.982
in 3 epoch, average loss: 450.5000305175781
                , loss1: -1.3203814029693604
                , loss2: 488.7593994140625
                , weight: 28.976
in 4 epoch, average loss: 45.01334762573242
                , loss1: -1.8833003044128418
                , loss2: 99.57255554199219
                , weight: 28.97
in 5 epoch, average loss: 40.56822967529297
                , loss1: -1.5904115438461304
                , loss2: 86.6329116821289
                , weight: 28.964
in 6 epoch, avera

KeyboardInterrupt: 

In [21]:
hgnn_trainer.eval()
outs = hgnn_trainer.forward(hgnn_trainer.X)
outs_straight = StraightThroughEstimator.apply(outs)
G_clone = G.clone()
edges, _  = G_clone.e
cut = 0
for vertices in edges:
    if torch.prod(outs_straight[list(vertices)], dim=0).sum() == 0:
        cut += 1
    else:
        G_clone.remove_hyperedges(vertices)
assert cut == G_clone.num_e
print(cut)

236


In [31]:
num_nodes = outs_straight.sum(dim=0)
print(num_nodes.sum())
print(num_nodes)


tensor(1242., device='cuda:1', grad_fn=<SumBackward0>)
tensor([349., 438., 455.], device='cuda:1', grad_fn=<SumBackward1>)
